In [1]:
from hydra.utils import instantiate
import torch
import torch.nn as nn
from omegaconf import DictConfig, OmegaConf
import hydra
from detector.data.dataset import ThousandLandmarksDataset

/home/xrenya/anaconda3/envs/ml/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conf = OmegaConf.load('configs/train.yaml')

In [12]:
data = {'root': 'data/train/', 'transforms': None, 'split': 'train', 'train_test_split': 0.5}

In [13]:
import os
from abc import ABC, abstractmethod
from typing import Callable, Dict, List, Optional, OrderedDict, Tuple
from sklearn.model_selection import train_test_split
import albumentations as A
import cv2
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset

import cv2
import tqdm
import numpy as np
import albumentations as A
import torch
from torch.utils.data import Dataset


class DetectionDataset(ABC, Dataset):
    @abstractmethod
    def __init__(self):
        super().__init__()
        self.name = type(self).__name__

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        sample = {}
        if self.keypoints:
            sample["keypoints"] = self.df.iloc[1:, index]
        image = cv2.imread(self.df.iloc[0, index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        sample["image"] = image

        if self.transforms is not None:
            sample = self.transforms(**sample)

        return sample

def get_dataframes(
        landmarks: str,
        landmarks_train: str,
        landmarks_val: str,
        landmarks_test: str,
        seed: int,
        train_size: float,
        shuffle: bool
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df = pd.read_csv(landmarks)
    train_df, valid_df = train_test_split(df, train_size=train_size, random_state=seed, shuffle=shuffle)
    train_df.to_csv(landmarks_train)
    valid_df.to_csv(landmarks_val)

    test_df = pd.read_csv(landmarks_test)

    logging.info(f"Train dataset: {len(train_df)}")
    logging.info(f"Valid dataset: {len(valid_df)}")
    logging.info(f"Test dataset: {len(test_df)}")

    return train_df, valid_df, test_df


def get_dataset(config: Config) -> Tuple[Dataset, Dataset, Dataset]:
    df_train, df_val, df_test = get_dataframes(config)

    train_dataset = AmazonDataset(df_train,
                                  config,
                                  augmentation=config.augmentations,
                                  preprocessing=config.preprocessing)

    valid_dataset = AmazonDataset(df_train,
                                  config,
                                  augmentation=None,
                                  preprocessing=config.preprocessing)

    test_dataset = AmazonDataset(df_test,
                                 config,
                                 augmentation=None,
                                 preprocessing=config.preprocessing)

    return train_dataset, valid_dataset, test_dataset


def get_loaders(dataframe):
    train_dataset, valid_dataset, test_dataset = dataframe

    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        worker_init_fn=worker_init_fn,
    )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        worker_init_fn=worker_init_fn,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        worker_init_fn=worker_init_fn,
    )

    return OrderedDict({
        "train": train_loader,
        "valid": valid_loader
    }), {
        "infer": test_loader
    }


class ThousandLandmarksDataset(DetectionDataset):
    def __init__(self, df: pd.DataFrame, split: str):
        super(ThousandLandmarksDataset, self).__init__()
        self.df = df
        self.keypoints = 0
        if split in ("train", "val"):
            self.keypoints = 1

/home/xrenya/anaconda3/envs/ml/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'Config' is not defined

In [14]:
dataset = ThousandLandmarksDataset(**data)

  0%|          | 0/64001 [00:00<?, ?it/s]/home/xrenya/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:65: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/xrenya/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:66: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review y

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/train/landmarks.csv', delimiter="\t")

In [5]:
df

,file_name,Point_0_X,Point_0_Y,Point_1_X,Point_1_Y,Point_2_X,Point_2_Y,Point_3_X,Point_3_Y,Point_4_X,...,Point_966_X,Point_966_Y,Point_967_X,Point_967_Y,Point_968_X,Point_968_Y,Point_969_X,Point_969_Y,Point_970_X,Point_970_Y
0,07c5e48d938e8d82c0f10973d66183f3.jpg,162,311,159,311,156,311,152,311,149,...,125,254,124,254,123,254,112,163,191,166
1,28e70684d6e2d6d75a6d3a973e4d4d93.jpg,90,376,89,375,85,374,84,374,82,...,87,307,85,309,87,310,96,202,162,203
2,e7696776dc7b15ca3fa80d4332443e96.jpg,123,210,121,210,119,211,117,211,115,...,98,175,97,175,95,176,80,119,128,115
3,199668f481aa5794c28657bace02fd84.jpg,109,203,107,203,105,203,103,203,101,...,75,159,73,158,73,157,75,107,129,106
4,9ebb7d722fa1cece4fa0d7b2fd39b399.jpg,59,127,58,126,56,126,55,126,53,...,44,92,43,91,43,91,58,61,84,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63995,2230b6b73ebe92d8f00f8e6d00b318d5.jpg,182,374,178,374,175,374,171,373,168,...,139,290,136,288,137,288,148,201,236,206
63996,1fe7f1bac9a008a2cac17e5e5e81aa76.jpg,225,465,222,465,218,465,215,465,211,...,168,377,165,376,164,375,142,274,256,253
63997,e83b11c7dabf9af60ebe1b9b8e0d7915.jpg,142,342,139,342,136,341,133,341,130,...,117,271,115,269,116,268,134,188,217,204
63998,5f30bdb45bdcb0588da9094509dba903.jpg,66,148,65,148,64,148,63,148,62,...,53,121,52,121,52,120,45,84,81,82


In [9]:
train_df, valid_df = train_test_split(df, train_size=0.1, random_state=0)

In [54]:
class DetectionDataset(ABC, Dataset):
    @abstractmethod
    def __init__(self):
        super().__init__()
        self.name = type(self).__name__
        self.images = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        sample = {}
        if self.keypoints:
            sample["keypoints"] = np.array(self.df.iloc[index, 1:])
        image_path = os.path.join(self.images, self.df.iloc[index, 0])
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        sample["image"] = image

        if self.transforms is not None:
            sample = self.transforms(**sample)

        return sample

def get_dataframes(
        landmarks: str,
        landmarks_train: str,
        landmarks_val: str,
        landmarks_test: str,
        seed: int,
        train_size: float,
        shuffle: bool
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df = pd.read_csv(landmarks)
    train_df, valid_df = train_test_split(df, train_size=train_size, random_state=seed, shuffle=shuffle)
    train_df.to_csv(landmarks_train)
    valid_df.to_csv(landmarks_val)

    test_df = pd.read_csv(landmarks_test)

    logging.info(f"Train dataset: {len(train_df)}")
    logging.info(f"Valid dataset: {len(valid_df)}")
    logging.info(f"Test dataset: {len(test_df)}")

    return train_df, valid_df, test_df

class ThousandLandmarksDataset(DetectionDataset):
    def __init__(self, df: pd.DataFrame, image_path: str, split: str, transforms: Optional[A.Compose] = None):
        super(ThousandLandmarksDataset, self).__init__()
        self.df = df
        self.images = image_path
        self.keypoints = 0
        self.transforms = transforms
        if split in ("train", "val"):
            self.keypoints = 1

In [55]:
dataloader = ThousandLandmarksDataset(train_df, "data/train/images", "train")

In [59]:
keys = dataloader[0]['keypoints']

data/train/images/50529e125aad2c470669b493bd1eac22.jpg


In [116]:
cv2.imread("/home/xrenya/Documents/MADE/DL/CV1/data/train/images/ea7dd5a70c71c1a5f61c37ddafb0266b.jpg")

array([[[ 0,  0,  0],
        [ 1,  1,  1],
        [ 0,  0,  1],
        ...,
        [ 0,  0,  1],
        [ 0,  0,  1],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 1,  0,  2],
        ...,
        [ 4,  3,  5],
        [ 2,  1,  3],
        [ 0,  0,  0]],

       [[ 2,  1,  3],
        [ 0,  0,  1],
        [ 0,  0,  1],
        ...,
        [ 0,  0,  1],
        [ 0,  0,  1],
        [ 1,  1,  1]],

       ...,

       [[ 7,  7, 13],
        [ 8,  8, 14],
        [12, 12, 18],
        ...,
        [11, 21, 39],
        [ 0,  0,  8],
        [ 0,  1,  2]],

       [[10, 10, 16],
        [ 6,  6, 12],
        [10, 10, 16],
        ...,
        [31, 44, 66],
        [ 0,  0, 10],
        [ 0,  0,  3]],

       [[ 9, 10, 14],
        [ 9, 10, 14],
        [11, 12, 16],
        ...,
        [60, 74, 97],
        [ 0,  0, 11],
        [ 0,  0,  3]]], dtype=uint8)

In [115]:
img

array([[[ 0,  0,  0],
        [ 1,  1,  1],
        [ 0,  0,  1],
        ...,
        [ 0,  0,  1],
        [ 0,  0,  1],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 1,  0,  2],
        ...,
        [ 4,  3,  5],
        [ 2,  1,  3],
        [ 0,  0,  0]],

       [[ 2,  1,  3],
        [ 0,  0,  1],
        [ 0,  0,  1],
        ...,
        [ 0,  0,  1],
        [ 0,  0,  1],
        [ 1,  1,  1]],

       ...,

       [[ 7,  7, 13],
        [ 8,  8, 14],
        [12, 12, 18],
        ...,
        [11, 21, 39],
        [ 0,  0,  8],
        [ 0,  1,  2]],

       [[10, 10, 16],
        [ 6,  6, 12],
        [10, 10, 16],
        ...,
        [31, 44, 66],
        [ 0,  0, 10],
        [ 0,  0,  3]],

       [[ 9, 10, 14],
        [ 9, 10, 14],
        [11, 12, 16],
        ...,
        [60, 74, 97],
        [ 0,  0, 11],
        [ 0,  0,  3]]], dtype=uint8)

In [89]:
df[df["file_name"] == "ea7dd5a70c71c1a5f61c37ddafb0266b.jpg"]

,file_name,Point_0_X,Point_0_Y,Point_1_X,Point_1_Y,Point_2_X,Point_2_Y,Point_3_X,Point_3_Y,Point_4_X,...,Point_966_X,Point_966_Y,Point_967_X,Point_967_Y,Point_968_X,Point_968_Y,Point_969_X,Point_969_Y,Point_970_X,Point_970_Y
52019,ea7dd5a70c71c1a5f61c37ddafb0266b.jpg,41,117,41,116,40,116,39,115,38,...,44,87,44,86,44,85,62,61,86,83


In [102]:
for root, dir, files in os.walk("data/train/images/"):
    print(files)

['530695d0246c149b5dc1bddf94373f32.jpg', '8de58edb1d27b537ed88d4eb1f75621d.jpg', '46ad02a627f34122ce72065a343d402b.jpg', '73267b0a1255e2ca88f0d8e453099639.jpg', 'c94021549217d1ea9bc52033b2b56055.jpg', 'b5850c3a40183a1b4f13d183c2ef47d2.jpg', 'dd936fbcd2cb8ebde37916bba9fb8d31.jpg', 'afcfb2a18d62d6e399bef20a1157e496.jpg', '4e9b38298287c46dcad7a5034b9ac81f.jpg', '745f768b84b68468988519027dfb9cd7.jpg', 'fc1dd92659f6c2767377edc85b7c247f.jpg', '45dd42d588599a97bc4ca6c7e0314906.jpg', 'fa243c43982e8e81e4ed82bb1453cbc6.jpg', 'e99a9c197b55b9a6de82fdd2effa4879.jpg', 'a7a5da77020b6f0acad68e039f10a68a.jpg', '219424950f89eaa11a9c4315c64abe8c.jpg', '8de7bbc2f21ee8de1d3e852d9ee56e96.jpg', '45b889f2938abf10189c5c722b117eef.jpg', 'be4c8865b062ecb2e4e21d436035eb2e.jpg', '55c299e1b7913e818e82fe19f9f7f3cc.jpg', 'c9a5243cc8e8eb522ce385cda44e3ae7.jpg', '20453dad7942c1394916e81be94be5b9.jpg', '1625b67b6f6e4a8f61018d8bc598cc4e.jpg', 'af1994520b952f0b6bd5040500b5107b.jpg', '26df73106325acdab63ca80db9f89f57.jpg',

In [105]:
root

'data/train/images/'

In [48]:
import logging
import os
from typing import Callable, Dict, List, Optional, OrderedDict, Tuple

import albumentations as A
import cv2
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset


class AmazonDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        config: Config,
        augmentation: Optional[A.Compose] = None,
        preprocessing: Optional[Callable] = None,
    ):
        self.df = df
        self.config = config
        self.augmentation = augmentation
        self.preprocessing = preprocessing

    def __getitem__(self, idx: int) -> Dict[str, np.ndarray]:
        row = self.df.iloc[idx]

        img_path = f"{os.path.join(self.config.images_dir, row.Id)}.jpg"
        targets = np.array(row.drop(["Id"]), dtype="float32")

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.augmentation is not None:
            image = self.augmentation(image=image)["image"]
        if self.preprocessing is not None:
            image = self.preprocessing(image)

        return {IMAGES: image, TARGETS: targets}

    def __len__(self) -> int:
        return len(self.df)


def split_dataset(config: Config) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    train_df = pd.read_csv(config.train_dataset_path)
    valid_df = pd.read_csv(config.valid_dataset_path)
    test_df = pd.read_csv(config.test_dataset_path)

    logging.info(f"Train dataset: {len(train_df)}")
    logging.info(f"Valid dataset: {len(valid_df)}")
    logging.info(f"Test dataset: {len(test_df)}")

    return train_df, valid_df, test_df


def get_class_names(config: Config) -> List[str]:
    class_names = list(
        pd.read_csv(config.train_dataset_path, nrows=1).drop("Id", axis=1))
    logging.info(f"Classes num: {len(class_names)}")
    return class_names


def get_dataset(config: Config) -> Tuple[Dataset, Dataset, Dataset]:
    df_train, df_val, df_test = _get_dataframes(config)

    train_dataset = AmazonDataset(df_train,
                                  config,
                                  augmentation=config.augmentations,
                                  preprocessing=config.preprocessing)

    valid_dataset = AmazonDataset(df_train,
                                  config,
                                  augmentation=None,
                                  preprocessing=config.preprocessing)

    test_dataset = AmazonDataset(df_test,
                                 config,
                                 augmentation=None,
                                 preprocessing=config.preprocessing)

    return train_dataset, valid_dataset, test_dataset


def get_loaders(
    config: Config
) -> Tuple[OrderedDict[str, DataLoader], OrderedDict[str, DataLoader]]:
    train_dataset, valid_dataset, test_dataset = get_dataset(config)

    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        worker_init_fn=worker_init_fn,
    )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        worker_init_fn=worker_init_fn,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        worker_init_fn=worker_init_fn,
    )

    return OrderedDict({
        "train": train_loader,
        "valid": valid_loader
    }), {
        "infer": test_loader
    }


In [143]:
CROP_SIZE = 128
from albumentations.pytorch.transforms import ToTensorV2

transform = A.Compose([
    A.SmallestMaxSize(max_size=CROP_SIZE, p=1., always_apply=True),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.CoarseDropout(
        max_holes=8,
        max_height=10,
        max_width=10,
        p=0.5,
        min_holes=2,
        min_height=5,
        min_width=5,
    ),
    A.CenterCrop(height=CROP_SIZE, width=CROP_SIZE, p=1.),
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
        max_pixel_value=255.0,
        always_apply=True,
        p=1.0
    ),
    ToTensorV2()
], keypoint_params=A.KeypointParams(format='xy'))

In [144]:
A.save(transform, 'configs/data/train_transform.json')

In [145]:
CROP_SIZE = 128
from albumentations.pytorch.transforms import ToTensorV2

transform = A.Compose([
    A.SmallestMaxSize(max_size=CROP_SIZE, p=1., always_apply=True),
    A.CenterCrop(height=CROP_SIZE, width=CROP_SIZE, p=1.),
    A.Normalize(
        mean=(0.485, 0.456, 0.406),
        std=(0.229, 0.224, 0.225),
        max_pixel_value=255.0,
        always_apply=True,
        p=1.0
    ),
    ToTensorV2()
], keypoint_params=A.KeypointParams(format='xy'))

In [146]:
A.save(transform, 'configs/data/transform.json')

In [125]:
def draw_landmarks(image, landmarks):
    for point in landmarks:
        x, y = point.astype(np.int)
        cv2.circle(image, (x, y), 1, (128, 0, 128), 1, -1)
    return image

,file_name,Point_0_X,Point_0_Y,Point_1_X,Point_1_Y,Point_2_X,Point_2_Y,Point_3_X,Point_3_Y,Point_4_X,...,Point_966_X,Point_966_Y,Point_967_X,Point_967_Y,Point_968_X,Point_968_Y,Point_969_X,Point_969_Y,Point_970_X,Point_970_Y
0,07c5e48d938e8d82c0f10973d66183f3.jpg,162,311,159,311,156,311,152,311,149,...,125,254,124,254,123,254,112,163,191,166
1,28e70684d6e2d6d75a6d3a973e4d4d93.jpg,90,376,89,375,85,374,84,374,82,...,87,307,85,309,87,310,96,202,162,203
2,e7696776dc7b15ca3fa80d4332443e96.jpg,123,210,121,210,119,211,117,211,115,...,98,175,97,175,95,176,80,119,128,115
3,199668f481aa5794c28657bace02fd84.jpg,109,203,107,203,105,203,103,203,101,...,75,159,73,158,73,157,75,107,129,106
4,9ebb7d722fa1cece4fa0d7b2fd39b399.jpg,59,127,58,126,56,126,55,126,53,...,44,92,43,91,43,91,58,61,84,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63995,2230b6b73ebe92d8f00f8e6d00b318d5.jpg,182,374,178,374,175,374,171,373,168,...,139,290,136,288,137,288,148,201,236,206
63996,1fe7f1bac9a008a2cac17e5e5e81aa76.jpg,225,465,222,465,218,465,215,465,211,...,168,377,165,376,164,375,142,274,256,253
63997,e83b11c7dabf9af60ebe1b9b8e0d7915.jpg,142,342,139,342,136,341,133,341,130,...,117,271,115,269,116,268,134,188,217,204
63998,5f30bdb45bdcb0588da9094509dba903.jpg,66,148,65,148,64,148,63,148,62,...,53,121,52,121,52,120,45,84,81,82


In [ ]:
image = cv2.imread(image_names[idx])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = draw_landmarks(image, landmarks[idx])

plt.subplot(NUM_ROWS, NUM_COLS, i)
plt.imshow(image)